In [ ]:
#%pip install python-decouple

In [1]:
#import libraries
from decouple import config
from ibm_watsonx_ai import APIClient
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.foundation_models.prompts import PromptTemplate, PromptTemplateManager
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
from ibm_watsonx_ai.foundation_models.schema import TextGenParameters
import pandas as pd
from sklearn.metrics import classification_report 
import import_ipynb
import sys
sys.path.append('../')
from Preprocessing import Train_Test_split as ts

Training data shape: (452068, 3)
Test data shape: (113018, 3)
Training samples: 361654, Validation samples: 90414, Test samples: 113018


In [2]:
#get the API key and project key
WX_API_KEY = config('WX_API_KEY')
PROJECT_ID = "f79c2d38-7ee2-4de6-931a-dad71b72d34f"

In [3]:
credentials = Credentials(
    url = "https://us-south.ml.cloud.ibm.com",
    api_key = WX_API_KEY
)

client = APIClient(
    credentials=credentials, 
    project_id=PROJECT_ID
)

In [ ]:
'''PARAMS = TextGenParameters(
    temperature=0.2,      # Higher temperature means more randomness
    max_new_tokens=3, # Maximum number of tokens to generate
    min_new_tokens=1, # Minimum number of tokens to generate
)

model = ModelInference(
    api_client=client,
    model_id="mistralai/mistral-small-3-1-24b-instruct-2503",
    params=PARAMS
)
TextGenParameters.show()'''

In [ ]:
PARAMS = TextGenParameters(
    temperature=0.2,      # Higher temperature means more randomness
    max_new_tokens=2, # Maximum number of tokens to generate
    min_new_tokens=1, # Minimum number of tokens to generate
)

model = ModelInference(
    api_client=client,
    model_id="mistralai/mistral-small-3-1-24b-instruct-2503",
    params=PARAMS
)
TextGenParameters.show()

+-----------------------+----------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------+
| PARAMETER             | TYPE                                   | EXAMPLE VALUE                                                                                                                             |
+=======================+========================================+===========================================================================================================================================+
| decoding_method       | str, TextGenDecodingMethod, NoneType   | sample                                                                                                                                    |
+-----------------------+----------------------------------------+----------------------------------------------------------------------------------------------------------

In [12]:
#This is partly generated using Copilot suggestions

def sentiment_prompt(text):
    prompt = """ 
    Return only a number based on the sentiment for the text. Do not give an explanation. Your response should only be a single character:
    1 for Negative
    2 for Neural
    3 for Positive.

    text: {}

    """
    return prompt.format(text).replace('\n', '')

In [13]:
#This is also generated using Copilot suggestions

def get_sentiment_llm(review):
    prompt = sentiment_prompt(review)
    response = model.generate_text(prompt)
    return response

In [14]:
df = ts.BI_df[["business_id", "text", "city", "date", "stars_3_scale", "address"]]
df

,business_id,text,city,date,stars_3_scale,address
31,aJvxWyQIG5OLfBw3qAe8xA,"Ordered Caramel frappe at Drive thru, BIG MIST...",Nashville,2017-12-29 19:38:31,1,2308 Murfreesboro Pike
46,MjZQqZAmJeMco_Vq-Y9h-g,Drum-roll please! Review #100 coming right up!...,Carmel,2014-02-05 19:38:24,3,13170 North Hazel Dell Pkwy
54,u7MJKcNdZXYyTeb67vD5jw,We stopped here for my Chai and Hubby's coffee...,Mount Laurel,2017-02-09 04:35:39,3,5 Hartford Rd
89,saJFbz12EnzanelpD8_xXQ,There's been three times that I've ordered a g...,Riverview,2016-08-25 14:08:18,1,10716 Big Bend Rd
104,KiE0h68HGOO7ZXAqkMBdiw,"I went in when they had 4 people working, wait...",Santa Barbara,2016-01-30 01:10:42,1,1015 De La Vina St
...,...,...,...,...,...,...
586567,-85kJMtb9wqNWDT8yLbitw,Fair service- messed up my drink cuppa times....,Franklin,2021-12-13 22:08:14,2,"438 Main St, Space 168, Space 168"
586568,Ddg-J_j0YFErk7wpMtH_0A,On my way out of Reno last week I noted a new ...,Reno,2021-12-28 19:26:37,3,986 Ambassador Dr
586638,2rmpfdyV2POqpXtmPHO_IQ,always closing drive thru. i've driven here so...,Tucson,2021-12-28 18:12:04,1,6363 N La Cholla
586646,GxuxCctcz3Hyk0wnuly7vQ,This is now one of two Starbucks in the shoppi...,Tucson,2017-01-21 00:19:27,3,4811 E Grant


In [15]:
df1 = df.iloc[0:100].copy()
df1["llm_sentiment"] = df1["text"].apply(get_sentiment_llm)

In [16]:
df1

,business_id,text,city,date,stars_3_scale,address,llm_sentiment
31,aJvxWyQIG5OLfBw3qAe8xA,"Ordered Caramel frappe at Drive thru, BIG MIST...",Nashville,2017-12-29 19:38:31,1,2308 Murfreesboro Pike,1
46,MjZQqZAmJeMco_Vq-Y9h-g,Drum-roll please! Review #100 coming right up!...,Carmel,2014-02-05 19:38:24,3,13170 North Hazel Dell Pkwy,3
54,u7MJKcNdZXYyTeb67vD5jw,We stopped here for my Chai and Hubby's coffee...,Mount Laurel,2017-02-09 04:35:39,3,5 Hartford Rd,3
89,saJFbz12EnzanelpD8_xXQ,There's been three times that I've ordered a g...,Riverview,2016-08-25 14:08:18,1,10716 Big Bend Rd,1
104,KiE0h68HGOO7ZXAqkMBdiw,"I went in when they had 4 people working, wait...",Santa Barbara,2016-01-30 01:10:42,1,1015 De La Vina St,1
...,...,...,...,...,...,...,...
3295,mBcvp7NarEQEviJyRiX23Q,Always a pleasure to stop in while waiting for...,Philadelphia,2017-03-16 00:54:47,3,57-63 North Third St,3
3299,BauybYsfqd0y6KDrJ6ZxjQ,I have been to this Starbucks multiple times a...,Tucson,2016-08-05 14:47:00,1,"7288 E Broadway Blvd, Ste 150",1
3304,RsJE0PIwTgKSB6CHmutjAw,Great location! They are always open on time (...,Edmonton,2014-07-14 01:33:13,3,"Northtown Mall, 9404 137 Ave NW",3
3386,GphUtL7kAHxd1CXxntt-Jw,This is a really good starbucks. One thing I l...,Bordentown,2017-08-29 14:14:51,3,282 Dunns Mill Rd,3


In [ ]:
df2 = df.iloc[5000:10000].copy()
df2["llm_sentiment"] = df2["text"].apply(get_sentiment_llm)

In [ ]:
df3 = df.iloc[10000:15000].copy()
df3["llm_sentiment"] = df3["text"].apply(get_sentiment_llm)

In [ ]:
df4 = df.iloc[15000:].copy()
df4["llm_sentiment"] = df4["text"].apply(get_sentiment_llm)

In [ ]:
final_df = pd.concat([df1,df2,df3,df4])

In [ ]:
final_df.info()

In [ ]:
final_df.head(10)

In [ ]:
%pip install pyarrow

In [ ]:
#final_df.to_parquet('llm_sentiment.parquet')
final_df.to_parquet('llm_sentiment_temp0_2.parquet')

In [ ]:
#final_df.to_json("llm_sentiment.json", orient="records", lines=True)
final_df.to_json("llm_sentiment_temp0_2.json", orient="records", lines=True)